In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split


In [5]:
data = pd.read_csv('train.csv')

X = data.drop('label',axis=1).values
y= data['label'].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32) / 255.0
X_test  = torch.tensor(X_test, dtype=torch.float32)  / 255.0

y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)


input_size = 784
output_size = 10

First Model, Linear

In [ ]:
import torch.nn as nn

model = nn.Linear(input_size,output_size)

# Training

learning_rate = 0.01
n_iters = 30


loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # forward
    y_pred = model(X_train)

    #loss
    l=loss(y_pred,y_train)

    #grad 
    l.backward()
    
    optimizer.step()

    optimizer.zero_grad()
    
    #steps
    if epoch %5 ==0 : 
        print(f'epoch {epoch +1}:, loss ={l.item():.4f} ')


with torch.no_grad():
    y_pred_test = model(X_test)
    predicted_labels = torch.argmax(y_pred_test, dim=1)

    accuracy = (predicted_labels == y_test).float().mean()
    print(f"Test accuracy = {accuracy.item():.4f}")


epoch 1:, loss =2.3146 
epoch 6:, loss =0.9268 
epoch 11:, loss =0.5893 
epoch 16:, loss =0.4731 
epoch 21:, loss =0.4190 
epoch 26:, loss =0.3858 
Test accuracy = 0.8936


# Drawing & Predicting

In [25]:
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("TkAgg")

canvas = np.zeros((28, 28), dtype=np.uint8)

fig, ax = plt.subplots()
im = ax.imshow(canvas, cmap="gray", vmin=0, vmax=255)
ax.set_title("Dessine un chiffre puis ferme la fenêtre")

def on_move(event):
    if event.button == 1 and event.inaxes:
        x, y = int(event.xdata), int(event.ydata)
        if 0 <= x < 28 and 0 <= y < 28:
            canvas[y, x] = 255   # <-- valeur visible
            im.set_data(canvas)
            fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_move)
plt.show()

# Préparation du tensor
img = torch.tensor(canvas, dtype=torch.float32).reshape(1, 784) / 255.0

with torch.no_grad():
    logits = model(img)
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()

print("Prediction:", pred)


Prediction: 1


Pretty bad model, let's improve it : MLP

In [39]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.network(x)

model2 = MLP()
# Training

learning_rate = 0.01
n_iters = 100


loss2= nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # forward
    y_pred = model2(X_train)

    #loss
    l=loss(y_pred,y_train)

    #grad 
    l.backward()
    
    optimizer.step()

    optimizer.zero_grad()
    
    #steps
    if epoch %5 ==0 : 
        print(f'epoch {epoch +1}:, loss ={l.item():.4f} ')


with torch.no_grad():
    y_pred_test = model2(X_test)
    predicted_labels = torch.argmax(y_pred_test, dim=1)

    accuracy = (predicted_labels == y_test).float().mean()
    print(f"Test accuracy = {accuracy.item():.4f}")


epoch 1:, loss =2.3033 
epoch 6:, loss =0.8185 
epoch 11:, loss =0.5547 
epoch 16:, loss =0.3772 
epoch 21:, loss =0.2998 
epoch 26:, loss =0.2599 
epoch 31:, loss =0.2252 
epoch 36:, loss =0.1963 
epoch 41:, loss =0.1726 
epoch 46:, loss =0.1528 
epoch 51:, loss =0.1360 
epoch 56:, loss =0.1220 
epoch 61:, loss =0.1095 
epoch 66:, loss =0.0984 
epoch 71:, loss =0.0885 
epoch 76:, loss =0.0796 
epoch 81:, loss =0.0715 
epoch 86:, loss =0.0640 
epoch 91:, loss =0.0573 
epoch 96:, loss =0.0511 
Test accuracy = 0.9636


In [ ]:
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("TkAgg")

canvas = np.zeros((28, 28), dtype=np.uint8)

fig, ax = plt.subplots()
im = ax.imshow(canvas, cmap="gray", vmin=0, vmax=255)
ax.set_title("Dessine un chiffre puis ferme la fenêtre")

def on_move(event):
    if event.button == 1 and event.inaxes:
        x, y = int(event.xdata), int(event.ydata)
        if 0 <= x < 28 and 0 <= y < 28:
            canvas[y, x] = 255   # <-- valeur visible
            im.set_data(canvas)
            fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_move)
plt.show()

# Préparation du tensor
img = torch.tensor(canvas, dtype=torch.float32).reshape(1, 784) / 255.0

with torch.no_grad():
    logits = model2(img)
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()

print("Prediction:", pred)


Prediction: 6
